In [274]:
#!/usr/bin/env python
from googleads import adwords
import pandas as pd
from StringIO import StringIO

In [283]:
class adwords_accounts(object):
    def __init__(self, start, end, version='v201705'):
        self._page_size = 500
        self._client = adwords.AdWordsClient.LoadFromStorage()
        self._version = version
        self._accounts = {}
        self._df_accounts = None
        self._start = start
        self._end = end
            
    def pull_accounts(self):
        mcc = self._client.GetService('ManagedCustomerService', version=self._version)
                    # Construct selector to get all accounts.
        offset = 0
        selector = {
            'fields': ['CustomerId', 'Name'],
            'predicates': [{'field' : 'ExcludeHiddenAccounts',
                            'operator' : 'EQUALS' ,
                            'values' : 'true'
                            }],
            'paging': {
                'startIndex': str(offset),
                'numberResults': str(self._page_size)
            }
        }
        more_pages = True
        while more_pages:
            # Get serviced account graph.
            page = mcc.get(selector)
            if 'entries' in page and page['entries']:
                for account in page['entries']:
                    self._accounts[account['customerId']] = account['name']
            offset += self._page_size
            selector['paging']['startIndex'] = str(offset)
            more_pages = offset < int(page['totalNumEntries'])
    
    def get_accounts(self, filter_old=True):
        if not self._accounts:
            self.pull_accounts()
        df = pd.DataFrame(self._accounts.items())
        df.columns = ['account_id', 'account_name']
        if filter_old:
            df = df[df.account_name.str.contains('^[a-z]{2}\_jobseeker\_usd\_.*', regex=True, na=False)]
            df = df[~df.account_name.str.contains('^.*\_aed\_.*', regex=True, na=False)]
            df = df[~df.account_name.str.contains('^.*\_app\_promo.*', regex=True, na=False)]
            df = df[~df.account_name.str.contains('^.*\_app\_active.*', regex=True, na=False)]
            df = df[~df.account_name.str.contains('^.*\_disabled.*', regex=True, na=False)]
            df = df[~df.account_name.str.contains('^.*\_display\_.*', regex=True, na=False)]
            df = df[~df.account_name.str.contains('^.*\_content\_.*', regex=True, na=False)]
            df = df[~df.account_name.str.contains('^.*\_dsa\_.*', regex=True, na=False)]
            df['country'] = df.account_name.apply(lambda x: x[0:2])
        self._df_accounts = df.sort_values('country')
    
    def get_all_countries(self):
        if self._df_accounts is None:
            self.get_accounts()   
        return self._df_accounts.groupby('country')['country'].count().to_dict()
    
    def get_non_eta_ads(self, account_id):
        self._client.SetClientCustomerId(account_id)
        report_downloader = self._client.GetReportDownloader(version=self._version)
        qry = ('SELECT CampaignId, AdGroupId, AdGroupStatus, Id, AdType, Impressions, Clicks, Cost FROM '
               'AD_PERFORMANCE_REPORT WHERE Status IN [ENABLED] '
               'AND AdGroupStatus in [ENABLED, PAUSED] DURING {0}, {1}').format(self._start, self._end)  
        stream_data = report_downloader.DownloadReportAsStringWithAwql(qry, 'CSV',
                skip_report_header=True, skip_column_header=False,
                skip_report_summary=True, include_zero_impressions=True)
        report_data = StringIO(stream_data)
        report_df = pd.DataFrame.from_csv(report_data, sep=',').reset_index()
        report_df = report_df.sort_values('Impressions', ascending=True)
        return report_df[report_df['Ad type']=='Text ad']


In [284]:
a = adwords_accounts('20170701', '20170801')

In [285]:
a.get_all_countries();

In [286]:
a.get_non_eta_ads(8676005523)

,Campaign ID,Ad group ID,Ad group state,Ad ID,Ad type,Impressions,Clicks,Cost


In [264]:
print a._df_accounts

     account_id                          account_name country
710  8676005523      ae_jobseeker_usd_broadmatch_0_en      ae
314  8327166874            ae_jobseeker_usd_main_0_en      ae
552  8306578974      ae_jobseeker_usd_exactmatch_0_en      ae
51   6715013252      ae_jobseeker_usd_exactmatch_1_en      ae
310  4167248807  ar_jobseeker_usd_broadmatch_0_active      ar
398  9714936970            ar_jobseeker_usd_main_0_es      ar
528  6072325590      ar_jobseeker_usd_exactmatch_0_es      ar
473  2035621172            at_jobseeker_usd_main_0_de      at
486  5174482260  at_jobseeker_usd_broadmatch_0_active      at
384  2123185258      at_jobseeker_usd_exactmatch_0_de      at
601  1844893370            au_jobseeker_usd_main_0_en      au
103  2171459886      au_jobseeker_usd_exactmatch_0_en      au
587  3137136265      au_jobseeker_usd_broadmatch_0_en      au
664  5676685164      au_jobseeker_usd_broadmatch_1_en      au
196  3408806462      be_jobseeker_usd_exactmatch_1_nl      be
179  426